In [1]:
import os.path
import sys
import math
from statistics import *
import numpy as np
from matplotlib import pyplot as plt

import skimage.io as io

from skimage.measure import *
from skimage import *
from skimage.transform import probabilistic_hough_line
from skimage.feature import *
from skimage.morphology import *
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean, rotate

In [2]:
def load_images(path, n):
    images = []
    suffix = '.png'
    for i in range(n):
        filepath = os.path.join(path, str(i) + suffix)
        images.append(io.imread(filepath, as_gray=True))
    return images

def load_test_ranking(path):
    f = open(os.path.join(path, "correct.txt"), "r")
    img_ranking = [int(a) for a in f.read().split()]
    
    f.close()
    return img_ranking

In [3]:
def swap(a, b):
    c = a
    a = b
    b = c

In [4]:
def distance(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return round(math.sqrt((x1-x2)**2 + (y1-y2)**2),3)

In [5]:
# usuwa niepotrzebne białe ramki;
# 0 - top, 1 - bottom, 2 - left, 3 - right
def drop_side(arr, side, n = 2): 
    if n <= 0:
        return arr
    
    _arr = arr
    while True:
        x, y = _arr.shape
        if side == 0:
            if n >= x: return _arr
            _to_sum = _arr[n]
            _next   = _arr[1:]
        elif side == 1:
            if n >= x: return _arr
            _to_sum = _arr[-n-1]
            _next   = _arr[:-1]
        elif side == 2:
            if n >= y: return _arr
            _to_sum = _arr[:,n]
            _next   = _arr[:,1:]
        elif side == 3:
            if n >= y: return _arr
            _to_sum = _arr[:,-n-1]
            _next   = _arr[:,:-1]
            
        if np.count_nonzero(_to_sum) == 0:
            _arr = _next
        else:
            return _arr
    
    return arr
    
def drop_empty_borders(arr, n = 2):
    a = drop_side(arr, 0, n)
    b = drop_side(a, 1, n)
    c = drop_side(b, 2, n)
    d = drop_side(c, 3, n)
    return d

In [6]:
def add_empty_borders(image):
    img_x = image.shape[0]    
    img_y = image.shape[1]
    result = np.zeros((img_x*3, img_y*3))
    _x = int(img_x)  
    _y = int(img_y)
    
    result[_x:_x+img_x, _y:_y+img_y] = image
    return result

In [7]:
def reverse(l):
    a = l
    a.reverse()
    return a

In [8]:
def binarize_image(img):
    return np.where (img>0.0, 1.0, 0.0)

In [9]:
def normalise_img_size(img, size = 1024):
    y_scale = size / img.shape[1]
    image_resized = resize(img, (round(y_scale * img.shape[0]), size), anti_aliasing=True)
    image_resized = np.where(image_resized > 0.0, 1.0, 0.0)
    return image_resized

In [10]:
# wyznacza punkt z listy, który jest najbliżej podanego
def nearest_point_to(point, points):
    distance_list = [(distance(point, p), p) for p in points]
    _points = [p for _, p in sorted(distance_list)]
    return _points[0]

In [11]:
# wyznacza kąt między 2 prostymi o wspólnym punkcie p1
def get_angle(p0, p1=np.array([0,0]), p2=None):
    if p2 is None:
        p2 = p1 + np.array([1, 0])
    v0 = np.array(p0) - np.array(p1)
    v1 = np.array(p2) - np.array(p1)

    angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
    return np.degrees(angle)

In [12]:
def tuple_swap(t):
    a, b = t
    return (b,a)

In [13]:
def angle_between_2_lines(a, b):
    p1, p2 = a
    x1 = p1[0]
    y1 = p1[1]
    x2 = p2[0]
    y2 = p2[1]
    a_angle = math.atan2((y1 - y2), (x1 - x2))

    p1, p2 = b
    x1 = p1[0]
    y1 = p1[1]
    x2 = p2[0]
    y2 = p2[1]
    b_angle = math.atan2((y1 - y2), (x1 - x2))
    result = math.degrees(abs(b_angle - a_angle)) % 180
    if(result > 90):
        return 180 - result
    else:
        return result

In [14]:
def distance_between_two_lines(lineA, lineB):
    (a1x, a1y), (a2x, a2y) = lineA
    (b1x, b1y), (b2x, b2y) = lineB
    center_a = ((a1x + a2x)/2), ((a1y + a2y)/2)
    center_b = ((b1x + b2x)/2), ((b1y + b2y)/2)
    return distance(center_a, center_b)

In [15]:
def take_longest_and_not_parallel_lines(lines):
    treshold = 5 #deg
    lines_ = sorted([(distance(p1,p2), p1, p2) for p1, p2 in lines])
    lines_ = [(p1, p2) for _, p1, p2 in lines_]
    lines_.reverse()
    
#     if len(lines) > 10:
#         lines_ = lines_[:int(len(lines_)/10)]
#     return lines_
    lines = lines_
    result = []
    while len(lines) > 0:
        l = lines[0]
        result.append(l)
        lines = lines[1:]
        if len(lines) > 0:
            l1, l2 = l    
            lines = [(p1, p2) for p1, p2 in lines if angle_between_2_lines((p1, p2), (l1, l2)) > treshold]

    return result

In [16]:
def degrees_between_line_and_x_axis(p1, p2):
    x1 = p1[0]
    y1 = p1[1]
    x2 = p2[0]
    y2 = p2[1]
    
    return -math.degrees(math.atan2((y1 - y2), (x1 - x2))) + 90

In [17]:
def calc_img_determinant(img):
    result = []
    for i in range(img.shape[1]):
        result.append(np.count_nonzero(img[:,i]))
    
    return result

In [18]:
def normalise_vector(v):
    _max = np.max(v)
    
    # zabezpieczenie: gdy są wszędzie takie same wartości to jest błąd (!)
    if np.all(v == _max):
        v = np.zeros(len(v))
        v += 0.5
        return list(v)
        
    _min = np.min(v)
    v = [ (x - _min)/(_max - _min) for x in v]
    return v

In [ ]:
def get_correctly_rotated_images(images):
    correctly_rotated_images = []

    for image in images:
        image = add_empty_borders(image)
        image = drop_empty_borders(image, 8)
        image = normalise_img_size(image, 256) 

        image_orig = image
        canny_border = canny(image)                         
        lines = probabilistic_hough_line(canny_border, threshold=24, line_length=16, line_gap=36)
        lines = take_longest_and_not_parallel_lines(lines)
        #coords = corner_peaks(corner_harris(image), min_distance=3)
        #points = list(zip(coords[:, 0], coords[:, 1]))

        (p1, p2) = lines[0]
        p = (p1, p2) = (tuple_swap(p1), tuple_swap(p2))

        theta = degrees_between_line_and_x_axis(p1, p2)

###################################################################################################
########################## RYSOWANIE FIGURY, PUNKTÓW ORAZ PODSTAWY ################################

#         fig, ax = plt.subplots()
#         plt.imshow(canny_border, cmap=plt.cm.gray)
#         plt.plot(coords[:, 1], coords[:, 0], color='red', marker='o', linestyle='None', markersize=5)

#         for line in lines[:1]:
#             p0, p1 = line
#             plt.plot((p0[0], p1[0]), (p0[1], p1[1]), linewidth=4)
#         plt.figure()
#         plt.show()

###################################################################################################

        image_orig = rotate(add_empty_borders(image_orig), theta)
        image_orig = drop_empty_borders(image_orig,1) 
        image_orig = normalise_img_size(image_orig, 256)
        image_orig = image_orig[4:-4, 4:-4]
        image_orig = opening(closing(image_orig, disk(3)), disk(8))
        
        # dodatkowe rotacje; jeśli wymagane
        if image_orig.shape[0] > image_orig.shape[1]:
            image_orig = rotate(image_orig, 90)
        
        if np.count_nonzero(image_orig[-16]) < (image_orig.shape[1]*0.85):
            image_orig = rotate(image_orig, 180)

        correctly_rotated_images.append(image_orig)

    return correctly_rotated_images


In [20]:
def add_rows_at_bottom(img, n):
    counter = 0
    while counter < n:
        img = np.vstack((
            img,
            np.ones(img.shape[1]))
        )
        counter += 1
    return img

def cut_image_bottom(image):
    while True:
        if np.count_nonzero(image[-1]) == image.shape[1]:
            image = image[:-1]
        else:
            break
    return image

def cut_images_bottom(images):
    return [cut_image_bottom(image) for image in images]

In [21]:
def count_zeros_in_columns(img):
    return list(np.sum(img == 0, axis=0))

def count_zeros_in_image(img):
    return np.sum(img == 0)

In [22]:
def get_image_determinant(img):
    det = count_zeros_in_columns(img)
    det = normalise_vector(det)
    return det

def make_image_from_determinant(determinants, n = 64):
    image = []
    for det in determinants:
        result = np.ones(n)
        i = 0
        while i < (det * n):
            result[i] = 0.0
            i += 1
        result = np.flip(result)
        np.where(result > 0, 1.0, 0.0)
        image.append(result)
    image = img_as_float(np.transpose(image))
    return image

def get_ranking(cut_images):
    results = []
    cut_images = [make_image_from_determinant(get_image_determinant(img)) for img in cut_images]
    
    for i in range(len(cut_images)):
        determinants = []
        res_imgs = []
        current_image = img_as_int(cut_images[i])
        np.where(current_image > 0, 1, 0)
        for j in range(len(cut_images)):
            if i != j:
                _image = img_as_int(rotate(cut_images[j], 180))
                np.where(_image > 0, 1, 0)
                images_result = (current_image + _image) - (current_image * _image)
                images_result = binarize_image(images_result)
                res_imgs.append(images_result)
                
############################################################################################
################## RYSOWANIE KAŻDY DO KAŻDEGO ############################################## 
#                 print("{} and {}".format(i, j))
#                 plt.imshow(images_result, cmap=plt.cm.gray)
#                 plt.figure()
#                 plt.show()
############################################################################################     

                det = count_zeros_in_columns(images_result)
                det = stdev(det)
                
                determinants.append((det, j))
            else:
                res_imgs.append(current_image)
                
        result = [img for _, img in sorted(determinants)]
        results.append(result)
        
############################################################################################
################## RYSOWANIE WYNIKU ######################################################## 
#         idx = result[0]
#         print("For {} best is {}".format(i, idx))
#         plt.imshow(res_imgs[idx], cmap=plt.cm.gray)
#         plt.figure()
#         plt.show()
############################################################################################     
       
    return results

In [23]:
def grade(result, test_set):
    points = []
    for i in range(len(result)):

        p = 1 / (result[i].index(test_set[i]) + 1)
        points.append(p)
        
        # print("Img. {:2d} -- {} pkt. -- correct: {:2d}, given ranking: {}...".format(i, round(p, 3), test_set[i], result[i][:7]))
    _sum = np.sum(points)
    print("Total: {} pkt. ({}%)".format(round(_sum, 3), round(_sum / len(points) * 100),3))
    return points

In [24]:
def print_ranking(ranking):
    for img_r in ranking:
        for e in img_r:
            print(e, end='')
        print("\n")

In [25]:
def main(test_images_path, img_n):
    if len(sys.argv) != 3:
        print("Invalid number of args! Abort.")
        exit()

#     _, path, n = sys.argv
    path, n = test_images_path, img_n

    img = load_images(path, int(n))
    img_ranking = load_test_ranking(path) # test only
    
    rot_images = get_correctly_rotated_images(img)
    cut_images = cut_images_bottom(rot_images)
    ranking = get_ranking(cut_images)
    
    grade(ranking, img_ranking)
    

In [28]:
# przypadki testowe
def run_all():
    ns = [6,20,20,20,20,200,200,20,100]
    for i in range(9):
        test_images_path = os.path.join(os.getcwd(), 'test', 'set' + str(i))
        img_n = ns[i]
        print("Running set {}: ".format(i))
        main(test_images_path, img_n)
        print()

In [30]:
run_all()

Running set 0: 
Total: 6.0 pkt. (100.0%)

Running set 1: 
Total: 20.0 pkt. (100.0%)

Running set 2: 
Total: 20.0 pkt. (100.0%)

Running set 3: 
Total: 20.0 pkt. (100.0%)

Running set 4: 
Total: 20.0 pkt. (100.0%)

Running set 5: 
Total: 193.582 pkt. (97.0%)

Running set 6: 
Total: 171.181 pkt. (86.0%)

Running set 7: 
Total: 9.936 pkt. (50.0%)

Running set 8: 
Total: 36.948 pkt. (37.0%)

